# Mixing it up - adventures with sparse matrices and statistical models

### Douglas Bates, Department of Statistics, University of Wisconsin-Madison

## Abstract

For more years than I care to admit, I have been developing algorithms and implementations to fit what statisticians call _mixed-effects_ models, most recently in [Julia](http://julialang.org).  These models are most useful when applied to very large data sets, which makes creating an effective, flexible implementation difficult. Evaluation of the log-likelihood to be maximized by the parameter estimates requires updating large structures, preferably in place, and some intricate linear algebra.  Efficiency is best achieved by taking into account the special structure of the matrices representing the model.  The Julia implementation can now out-perform earlier implementations that I and others have created, in some cases by orders of magnitude.  The Julia type system, multiple dispatch, and, most of all, the fact that one can achieve both flexibility and performance in a single language,have been fundamental to achieving this performance.  

## Mixed-effects models and penalized least squares

Mixed-effects models are similar to _linear models_ (LMs) or _generalized linear models_ (GLMs) in that fitting the model requires estimation of coefficients in a _linear predictor_ expression.  In a _linear mixed model_ (LMM) or a _generalized linear mixed model_ (GLMM) there are two types of coefficients, or _effects_; _fixed-effects_, which are associated with factors that have reproducible levels, such as male/female, and _random-effects_, which are associated with factors whose levels are a sample from a population, such as _subject_ or _item_.

Random-effects can be viewed as _nuisance parameters_ in that they account for sources of variability in the observations but are often not by themselves of interest.  Frequently the number of random effects grows with the number of observations (you observe a few observations on each of many subjects).  Because their estimates will be ill-defined they are _regularized_.  The model-fitting involves selecting the regularization parameters to maximize a log-likelihood.

### Linear predictors and model matrices

In an LMM the $n$-dimensional response, $\bf y$, is considered to be the realization of random variable $\mathcal{Y}$ with conditional distribution
$$
{\mathcal Y}|({\mathcal B} = \bf b) \sim {\mathcal N}({\bf X\beta + Z b},\sigma^2\bf I_n) .
$$
The _linear predictor_, $\bf X\beta+Z b$, is based on two, known model matrices; the dense matrix $\bf X$ of size $n\times p$, and sparse matrix $\bf Z$ of size $n\times q$.

For example, we define a model for the `InstEval` data set from the [lme4 package](https://github.com/lme4/lme4)

In [1]:
using HDF5,ReTerms,StatsBase
inst = h5open("/var/tmp/dat.h5","r") do io g2dict(io,"inst") end;
dump(inst)

Dict{Symbol,Any} len 5
  R: DataArrays.PooledDataArray{Int32,UInt8,1}(73421) Int32[1,2,1,2]
  S: DataArrays.PooledDataArray{Int32,UInt16,1}(73421) Int32[1,1,1,1]
  P: DataArrays.PooledDataArray{Int32,UInt8,1}(73421) Int32[14,5,14,12]
  D: DataArrays.PooledDataArray{Int32,UInt16,1}(73421) Int32[525,560,832,1068]
  Y: Array(Float32,(73421,)) Float32[5.0,2.0,5.0,3.0,2.0,4.0,4.0,5.0,5.0,4.0  …  5.0,5.0,5.0,3.0,5.0,3.0,4.0,5.0,1.0,3.0]


where `Y`, the response, is the evaluation by student, `S`, of instructor, `D`, in a course from department, `P`.  The service factor, `R`, indicates if the course was a service course or not.  We create random effects terms for student, instructor and department.

In [2]:
revec = [reterm(inst[s]) for s in [:S,:D,:P]];
for i in eachindex(revec) println(size(revec[i])) end

(73421,2972)
(73421,1128)
(73421,14)


The three sparse matrices derived from these terms are, in this case, the indicator columns for the 2972 students, for the 1128 instructors, and for the 14 departments.  The matrix $\bf Z$, of size $73421\times 4114$ is the horizontal concatenation of these three.  It is (comparatively) large but very sparse -- each row has only three nonzeros.

The fixed effects in the model will be the intercept term and an indicator of a service course. The intercept corresponds to a typical evaluation in a non-service course, after adjusting for the student, instructor and department.  The coefficient for `R` is the change in typical evaluation for service compared to non-service courses.


In [3]:
X = hcat(ones(length(inst[:R])),convert(Vector{Float64},inst[:R] .== 2));
size(X)

(73421,2)

### Distribution of the random effects

The unconditional distribution of the random effects is also multivariate normal
$$
{\mathcal B}\sim\mathcal{N}(\bf 0,\Sigma_{\theta})
$$
The positive-definite covariance matrix, $\Sigma$, depends on a small parameter vector $\theta$.  For scalar random -effects such as these (a scalar effect for each student, each instuctor and each department), the elements of $\mathcal B$ are independent within and between groups.  That is, $\Sigma$ is diagonal. Furthermore the variances of the effects are constant within group.  For technical reasons we write
$$
\Sigma_{\theta} = \sigma^2\Lambda_{\theta}\Lambda_{\theta}'
$$
where $\sigma^2$ is the same scale parameter as in the conditional distribution of $\mathcal Y$.  $\Lambda_{\theta}$ is the lower Cholesky factor of the relative covariance matrix, $\Sigma/\sigma^2$.

### Henderson's mixed-model equations

Since the 1970's it has been recognized that, given a value of the covariance parameters, $\theta$, the conditional estimate of the fixed-effects, $\widehat{\bf\beta}$, and the "BLUPS" of the random effects, $\tilde{\bf b}$ are the solution to "Henderson's mixed-model equations"
$$
\begin{bmatrix}
\bf X'X & \bf X'Z \\
\bf Z'X & \bf Z'Z+\Sigma_{\theta}^{-1}
\end{bmatrix}
\begin{bmatrix}
\widehat{\beta}\\
\tilde{\bf b}
\end{bmatrix}
=
\begin{bmatrix}
\bf X'y\\
\bf Z'y
\end{bmatrix}
$$

However, this solution by itself does not allow for direct evaluation of the log-likelihood.  In [Fitting Linear Mixed-effects Models using lme4](http://arxiv.org/abs/1406.5823) we show that it is convenient to rearranging these equations as
$$
\begin{bmatrix}
\bf \Lambda_{\theta}'Z'Z\Lambda_{\theta} + I & \bf \Lambda_{\theta}'Z'X \\
\bf X'Z\Lambda_\theta & \bf X'X
\end{bmatrix}
\begin{bmatrix}
\tilde{\bf u}\\
\widehat{\beta}
\end{bmatrix}
=
\begin{bmatrix}
\bf \Lambda_\theta'Z'y\\
\bf X'y
\end{bmatrix}
$$
corresponding to the  _pseudo data_ representation of the penalized least squares problem
$$
\rho^2(\theta)=
\min_{\beta,\bf u} 
\left\|
\begin{bmatrix}
\bf y\\
\bf 0
\end{bmatrix} -
\begin{bmatrix}
\bf Z\Lambda_\theta & \bf X \\
\bf I_q & \bf 0
\end{bmatrix}
\begin{bmatrix}
\bf u\\
\beta
\end{bmatrix}
\right\|^2 .
$$

In this formulation it is natural to form the Cholesky factor of the system matrix for which the upper left block is
$$
\bf L_\theta L_\theta' = \bf \Lambda_{\theta}'Z'Z\Lambda_{\theta} + I
$$
providing straightforward evaluation of
$$
\log\left(\left|\bf \Lambda_{\theta}'Z'Z\Lambda_{\theta} + I\right|\right) = 2\log\left(\left|\bf L_\theta\right|\right).
$$

The profiled log-likelihood can be written as a function of $\theta$ only
$$
-2\ell(\theta|{\bf y}) = -2\log(|\bf L_\theta|)+n\left(1+\frac{2\pi\rho^2(\theta)}n\right) .
$$

In [4]:
m1 = LMM(X,revec,inst[:Y]);
fieldnames(m1)

7-element Array{Symbol,1}:
 :trms 
 :A    
 :L    
 :lower
 :pars 
 :gp   
 :fit  

In [5]:
size(m1.A)

(4,4)

In [6]:
A = m1.A;
for j in 1:4, i in j:4 println(i,", ",j," ",size(A[i,j])," ",typeof(A[i,j])) end

1, 1 (2972,2972) Base.LinAlg.Diagonal{Float64}
2, 1 (1128,2972) Base.SparseMatrix.SparseMatrixCSC{Float64,Int32}
3, 1 (14,2972) Array{Float64,2}
4, 1 (3,2972) Array{Float64,2}
2, 2 (1128,1128) Base.LinAlg.Diagonal{Float64}
3, 2 (14,1128) Base.SparseMatrix.SparseMatrixCSC{Float64,Int32}
4, 2 (3,1128) Array{Float64,2}
3, 3 (14,14) Base.LinAlg.Diagonal{Float64}
4, 3 (3,14) Array{Float64,2}
4, 4 (3,3) Base.LinAlg.LowerTriangular{Float64,Array{Float64,2}}


In [7]:
L = m1.L
for j in 1:4, i in j:4 println(i,", ",j," ",size(L[i,j])," ",typeof(L[i,j])) end

1, 1 (2972,2972) Base.LinAlg.Diagonal{Float64}
2, 1 (1128,2972) Base.SparseMatrix.SparseMatrixCSC{Float64,Int32}
3, 1 (14,2972) Array{Float64,2}
4, 1 (3,2972) Array{Float64,2}
2, 2 (1128,1128) Base.LinAlg.LowerTriangular{Float64,Array{Float64,2}}
3, 2 (14,1128) Array{Float64,2}
4, 2 (3,1128) Array{Float64,2}
3, 3 (14,14) Base.LinAlg.LowerTriangular{Float64,Array{Float64,2}}
4, 3 (3,14) Array{Float64,2}
4, 4 (3,3) Base.LinAlg.LowerTriangular{Float64,Array{Float64,2}}


In [8]:
fit(m1,true);

f_1: 242074.39366, [1.0,1.0,1.0]
f_2: 245004.25385, [1.75,1.0,1.0]
f_3: 243120.91228, [1.0,1.75,1.0]
f_4: 242089.80485, [1.0,1.0,1.75]
f_5: 238618.67037, [0.25,1.0,1.0]
f_6: 241923.49173, [1.0,0.25,1.0]
f_7: 242040.48484, [1.0,1.0,0.25]
f_8: 240279.6813, [0.0,0.526088,0.475208]
f_9: 241303.18604, [0.0,1.11671,1.25398]
f_10: 238515.96151, [0.325,0.925,1.0]
f_11: 238727.58367, [0.356272,1.00427,0.832985]
f_12: 238751.90605, [0.372493,0.983047,1.0]
f_13: 238373.18283, [0.275129,0.871663,0.98288]
f_14: 238935.0279, [0.153782,0.847714,0.89802]
f_15: 238378.29187, [0.297217,0.868986,1.0545]
f_16: 238341.75427, [0.327602,0.820679,0.99938]
f_17: 238317.43087, [0.346982,0.776926,1.05713]
f_18: 238311.78636, [0.372166,0.724728,1.10473]
f_19: 238322.1477, [0.393995,0.679606,1.13311]
f_20: 238326.75849, [0.369695,0.738777,1.10716]
f_21: 238366.17778, [0.385395,0.727565,1.09961]
f_22: 238256.50803, [0.358984,0.718761,1.10482]
f_23: 238160.3682, [0.334948,0.702827,1.10723]
f_24: 238028.96884, [0.311

In [9]:
gc(); @time fit(LMM(X,revec,inst[:Y]));

LoadError: LoadError: syntax: local declaration in global scope
while loading In[9], in expression starting on line 1